# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [51]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file_path = Path("output_data/cities.csv")
Cities = pd.read_csv(csv_file_path)


# Display sample data
Cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ljusdal,61.8288,16.0913,274.82,96,75,0.83,SE,1731218553
1,1,Muroto-misakicho,33.2833,134.1500,293.15,71,100,2.86,JP,1731218555
2,2,Port elizabeth,-33.9180,25.5701,292.21,80,20,1.54,ZA,1731218080
3,3,Saipan,15.1355,145.7010,301.84,83,51,6.26,MP,1731218560
4,4,Hawaiian paradise park,19.5933,-154.9731,295.78,91,26,0.45,US,1731218562


In [63]:
cities_df = pd.DataFrame(Cities)
print(cities_df)

NameError: name 'Cities' is not defined

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    size = 'Humidity',
    color = 'Humidity',
    title = 'Map of Humidity of Cities',
    geo = True,
    tiles = 'OSM'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
cities_data_class_filtered1 = cities_data_class[cities_data_class['Max Temp'] >= 292 ]
cities_data_class_filtered2 = cities_data_class_filtered1[cities_data_class_filtered1['Wind Speed'] <= 15]
# Drop any rows with null values
cities_data_class_filtered3 = cities_data_class_filtered2.dropna

# Display sample data
cities_data_class_filtered3

<bound method DataFrame.dropna of      City_ID                    City      Lat       Lng  Max Temp  Humidity  \
1          1        Muroto-misakicho  33.2833  134.1500    293.15        71   
2          2          Port elizabeth -33.9180   25.5701    292.21        80   
3          3                  Saipan  15.1355  145.7010    301.84        83   
4          4  Hawaiian paradise park  19.5933 -154.9731    295.78        91   
6          6                Mabaruma   8.2000  -59.7833    296.00       100   
..       ...                     ...      ...       ...       ...       ...   
588      588             Kailua-kona  19.6406 -155.9956    297.35        90   
590      590           Talata mafara  12.5732    6.0597    292.03        29   
592      592                 Ha tinh  18.3333  105.9000    300.04        56   
594      594           Haiku-pauwela  20.9219 -156.3051    297.06        79   
596      596              Carutapera  -1.1950  -46.0200    298.80        84   

     Cloudiness  

In [61]:
print(type(cities_data_class_filtered3))

<class 'method'>


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df_filtered3.copy())

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel_Name'] = None

# Display sample data
print(hotel_df)

AttributeError: 'function' object has no attribute 'copy'

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)